In [1]:
import Utils.PTModel.Training as Training
import Utils.PTModel.Inference as Inference
import Utils.Data.DataLoading as DataLoading
import Utils.PTModel.Models as Models

import Utils.EvaluationMetrics.alphaScore as AS
import Utils.EvaluationMetrics.exampleBased as EB
import Utils.EvaluationMetrics.labelBased as LB

import numpy as np
import pandas as pd

MODELARCH = Models.TileEmbeddingVAEwMHA

In [2]:
crossValidationData = DataLoading.LoadCrossValTrainTestData("../data/unshuffled3x3tiles.csv", shuffle=True)

# order of games in the original papers table
ordering = ['supermariobros', 'kidicarus', 'megaman', 'loderunner', 'thelegendofzelda']

# translation for original papers table format
translation = ['SMB', 'Kid Icarus', 'Megaman', 'Lode Runner', 'LOZ']


In [3]:
print(f"--- Cross Validation Testing ---")

crossValDataframe = pd.DataFrame()

crossValDataframe.insert(0, "AS B 0.25 G 0.75", 0)
crossValDataframe.insert(0, "AS B 1.0 G 1.0", 0)
crossValDataframe.insert(0, "AS B 0.75 G 0.25", 0)

crossValDataframe.insert(0, "LB Accuracy", 0)
crossValDataframe.insert(0, "LB Recall", 0)
crossValDataframe.insert(0, "LB Precision", 0)

crossValDataframe.insert(0, "EB Accuracy", 0)
crossValDataframe.insert(0, "EB Recall", 0)
crossValDataframe.insert(0, "EB Precision", 0)
crossValDataframe.insert(0, "EB EMR", 0)

crossValDataframe.insert(0, "Game Name", "")

for i, gamename in enumerate(ordering):

    k = gamename
    v = crossValidationData[gamename]

    print(f"\n{k}")
    print(f"trainDataShape: {v['trainData'].shape} | testDataShape: {v['testData'].shape}")

    model = Training.TrainModel(v, 50, 32, MODELARCH, earlyStoppingRange=2)

    yPredImage, yPredText = Inference.ModelInference(model, v['testData'])
    yTruthText = np.array(v['testData']["encodedAffordances"].to_list())

    as1 = AS.alpha_score(yTruthText, yPredText, beta=0.75, gamma=0.25)
    as2 = AS.alpha_score(yTruthText, yPredText, beta=1.0, gamma=1.0)
    as3 = AS.alpha_score(yTruthText, yPredText, beta=0.25, gamma=0.75)

    print(f"\nAlpha Score beta = 0.75 | gamma = 0.25: {AS.alpha_score(yTruthText, yPredText, beta=0.75, gamma=0.25)}")
    print(f"Alpha Score beta = 1.0 | gamma = 1.0: {AS.alpha_score(yTruthText, yPredText, beta=1.0, gamma=1.0)}")
    print(f"Alpha Score beta = 0.25 | gamma = 0.75: {AS.alpha_score(yTruthText, yPredText, beta=0.25, gamma=0.75)}")

    ebAcc = EB.example_based_accuracy(yTruthText, yPredText)
    ebPre = EB.example_based_precision(yTruthText, yPredText)
    ebRec = EB.example_based_recall(yTruthText, yPredText)
    # ebEMR = EB.hamming_loss(yTruthText, yPredText)
    ebEMR = EB.EMR(yTruthText, yPredText)

    print(f"\nExample Based Accuracy: {EB.example_based_accuracy(yTruthText, yPredText)}")
    print(f"Example Based Precision: {EB.example_based_precision(yTruthText, yPredText)}")
    print(f"Example Based Recall: {EB.example_based_recall(yTruthText, yPredText)}")
    # print(f"Example Based EMR: {EB.hamming_loss(yTruthText, yPredText)}")
    print(f"Example Based EMR: {EB.EMR(yTruthText, yPredText)}")

    lbAcc = LB.accuracy_macro(yTruthText, yPredText)
    lbPre = LB.precision_macro(yTruthText, yPredText)
    lbRec = LB.recall_macro(yTruthText, yPredText)
    
    print(f"Label Based Accuracy: {LB.accuracy_macro(yTruthText, yPredText)}")
    print(f"Label Based Precision: {LB.precision_macro(yTruthText, yPredText)}")
    print(f"Label Based Recall: {LB.recall_macro(yTruthText, yPredText)}")

    crossValDataframe.loc[len(crossValDataframe)] = [translation[i], ebEMR, ebPre, ebRec, ebAcc, lbPre, lbRec, lbAcc, as1, as2, as3]

crossValDataframe.head()

--- Cross Validation Testing ---

supermariobros
trainDataShape: (19132, 7) | testDataShape: (860, 7)


/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/src/Utils/PTModel/Training.py:57: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  xTextbatch = torch.tensor(trainData.iloc[j:j+batchSize]["encodedAffordances"].tolist(), dtype=torch.float32).to(device)


Epoch 0: loss 6446.904340150763 | Val Text Loss: 638.85986328125
Epoch 1: loss 3357.151927731109 | Val Text Loss: 599.8651123046875
Epoch 2: loss 2200.616450587244 | Val Text Loss: 581.3018798828125
Epoch 3: loss 1445.5505973280074 | Val Text Loss: 647.9371948242188
Epoch 4: loss 955.980903395841 | Val Text Loss: 781.0946044921875
Early Stopping...

Alpha Score beta = 0.75 | gamma = 0.25: 0.7976240984302079
Alpha Score beta = 1.0 | gamma = 1.0: 0.5163343232923208
Alpha Score beta = 0.25 | gamma = 0.75: 0.7187102248621129

Example Based Accuracy: 0.6706732608476795
Example Based Precision: 0.680479884147644
Example Based Recall: 0.7697674632072449
Example Based EMR: 0.5755813953488372
Label Based Accuracy: 0.19879014790058136
Label Based Precision: 0.23308417201042175
Label Based Recall: 0.3055912256240845

kidicarus
trainDataShape: (19110, 7) | testDataShape: (882, 7)


/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/src/Utils/EvaluationMetrics/labelBased.py:31: RuntimeWarning: invalid value encountered in divide
  sum_precision=np.nansum(p_n/p_d)
/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/src/Utils/EvaluationMetrics/labelBased.py:46: RuntimeWarning: invalid value encountered in divide
  sum_recall=np.nansum(p_n/p_d)


Epoch 0: loss 6327.671158908602 | Val Text Loss: 544.7975463867188
Epoch 1: loss 3274.317837616273 | Val Text Loss: 458.0381164550781
Epoch 2: loss 2094.6017636710585 | Val Text Loss: 545.7286987304688
Epoch 3: loss 1349.7549261903284 | Val Text Loss: 627.30224609375
Early Stopping...

Alpha Score beta = 0.75 | gamma = 0.25: 0.7687270822420116
Alpha Score beta = 1.0 | gamma = 1.0: 0.5474070193818753
Alpha Score beta = 0.25 | gamma = 0.75: 0.7786799371398638

Example Based Accuracy: 0.5976379440665154
Example Based Precision: 0.6743197441101074
Example Based Recall: 0.627362072467804
Example Based EMR: 0.4943310657596372
Label Based Accuracy: 0.2025986760854721
Label Based Precision: 0.3008370101451874
Label Based Recall: 0.20908069610595703

megaman
trainDataShape: (18523, 7) | testDataShape: (1469, 7)


...

Epoch 0: loss 9597.260212145353 | Val Text Loss: 1163.5445556640625
Epoch 1: loss 6244.679814388877 | Val Text Loss: 1204.712890625
Epoch 2: loss 4989.802980121814 | Val Text Loss: 1233.6634521484375
Early Stopping...

Alpha Score beta = 0.75 | gamma = 0.25: 0.6811727386592605
Alpha Score beta = 1.0 | gamma = 1.0: 0.4308936448292763
Alpha Score beta = 0.25 | gamma = 0.75: 0.7497209061700159

Example Based Accuracy: 0.5448925263458957
Example Based Precision: 0.6475040316581726
Example Based Recall: 0.5720414519309998
Example Based EMR: 0.41515411523991813
Label Based Accuracy: 0.1308842897415161
Label Based Precision: 0.15271463990211487
Label Based Recall: 0.189843088388443

thelegendofzelda
trainDataShape: (18362, 7) | testDataShape: (1630, 7)


/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/src/Utils/EvaluationMetrics/labelBased.py:16: RuntimeWarning: invalid value encountered in divide
  sum_acc=np.nansum(p_n/p_d)
/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/src/Utils/EvaluationMetrics/labelBased.py:31: RuntimeWarning: invalid value encountered in divide
  sum_precision=np.nansum(p_n/p_d)
/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/src/Utils/EvaluationMetrics/labelBased.py:46: RuntimeWarning: invalid value encountered in divide
  sum_recall=np.nansum(p_n/p_d)


Epoch 0: loss 6310.539748560676 | Val Text Loss: 626.670166015625
Epoch 1: loss 3419.6013900278335 | Val Text Loss: 1078.1107177734375
Epoch 2: loss 2251.2132619399226 | Val Text Loss: 1021.7105712890625
Epoch 3: loss 1472.3926405424857 | Val Text Loss: 1666.78173828125
Epoch 4: loss 966.3637399175026 | Val Text Loss: 1572.61962890625
Epoch 5: loss 631.3277276866528 | Val Text Loss: 2083.755615234375
Epoch 6: loss 402.6784350298838 | Val Text Loss: 1849.3853759765625
Epoch 7: loss 251.0735319582843 | Val Text Loss: 1367.6168212890625
Epoch 8: loss 158.98413621590112 | Val Text Loss: 1735.51953125
Epoch 9: loss 94.1114466996143 | Val Text Loss: 1615.822021484375
Epoch 10: loss 59.561233020410306 | Val Text Loss: 2305.134765625
Epoch 11: loss 40.42320893699699 | Val Text Loss: 1759.3905029296875
Epoch 12: loss 29.771960972493535 | Val Text Loss: 2080.89208984375
Epoch 13: loss 23.862639645250834 | Val Text Loss: 1712.349853515625
Epoch 14: loss 21.39494060888523 | Val Text Loss: 1106.321

/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/src/Utils/EvaluationMetrics/labelBased.py:16: RuntimeWarning: invalid value encountered in divide
  sum_acc=np.nansum(p_n/p_d)
/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/src/Utils/EvaluationMetrics/labelBased.py:31: RuntimeWarning: invalid value encountered in divide
  sum_precision=np.nansum(p_n/p_d)
/home/surfytom/Projects/Dissertation/Repos/TileEmbeddingDissertation/src/Utils/EvaluationMetrics/labelBased.py:46: RuntimeWarning: invalid value encountered in divide
  sum_recall=np.nansum(p_n/p_d)


,Game Name,EB EMR,EB Precision,EB Recall,EB Accuracy,LB Precision,LB Recall,LB Accuracy,AS B 0.75 G 0.25,AS B 1.0 G 1.0,AS B 0.25 G 0.75
0,SMB,0.575581,0.680480,0.769767,0.670673,0.233084,0.305591,0.198790,0.797624,0.516334,0.718710
1,Kid Icarus,0.494331,0.674320,0.627362,0.597638,0.300837,0.209081,0.202599,0.768727,0.547407,0.778680
2,Megaman,0.404357,0.568206,0.941457,0.554338,0.304314,0.363527,0.295688,0.833087,0.395795,0.562707
3,Lode Runner,0.415154,0.647504,0.572041,0.544893,0.152715,0.189843,0.130884,0.681173,0.430894,0.749721
4,LOZ,0.000000,0.200673,0.486503,0.183727,0.134634,0.465955,0.107088,0.716935,0.181020,0.464084


In [4]:
pd.set_option("display.precision", 2)
crossValDataframe

,Game Name,EB EMR,EB Precision,EB Recall,EB Accuracy,LB Precision,LB Recall,LB Accuracy,AS B 0.75 G 0.25,AS B 1.0 G 1.0,AS B 0.25 G 0.75
0,SMB,0.58,0.68,0.77,0.67,0.23,0.31,0.20,0.80,0.52,0.72
1,Kid Icarus,0.49,0.67,0.63,0.60,0.30,0.21,0.20,0.77,0.55,0.78
2,Megaman,0.40,0.57,0.94,0.55,0.30,0.36,0.30,0.83,0.40,0.56
3,Lode Runner,0.42,0.65,0.57,0.54,0.15,0.19,0.13,0.68,0.43,0.75
4,LOZ,0.00,0.20,0.49,0.18,0.13,0.47,0.11,0.72,0.18,0.46


In [22]:
crossValDataframe.to_csv("test.csv", index=False)